# Local LLMs BLEU-metrics
This notebook is used for models evaluation using BLEU metric score. Results are reproducable.

In [ ]:
!pip install datasets

In [ ]:
import locale
locale.getpreferredencoding = lambda: "UTF-8"

In [ ]:
! pip install evaluate

# RedPajama-Chat-3B-v1

In [ ]:
import torch
import transformers
from transformers import AutoTokenizer, AutoModelForCausalLM

MIN_TRANSFORMERS_VERSION = '4.25.1'

# check transformers version
assert transformers.__version__ >= MIN_TRANSFORMERS_VERSION, f'Please upgrade transformers to version {MIN_TRANSFORMERS_VERSION} or higher.'

In [ ]:
model2 = AutoModelForCausalLM.from_pretrained("togethercomputer/RedPajama-INCITE-Chat-3B-v1", torch_dtype=torch.float16)
tokenizer = AutoTokenizer.from_pretrained("togethercomputer/RedPajama-INCITE-Chat-3B-v1")
model2 = model2.to('cuda:0')

generation_config.json:   0%|          | 0.00/111 [00:00<?, ?B/s]

tokenizer_config.json:   0%|          | 0.00/237 [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/2.11M [00:00<?, ?B/s]

special_tokens_map.json:   0%|          | 0.00/99.0 [00:00<?, ?B/s]

In [ ]:
from datasets import load_dataset
test = load_dataset("wikitext", "wikitext-2-raw-v1", split="test")#.select(range(100))
encodings = tokenizer("\n\n".join(test["text"]), return_tensors="pt")

Token indices sequence length is longer than the specified maximum sequence length for this model (288730 > 2048). Running this sequence through the model will result in indexing errors


## fp-16 precision

In [ ]:
import torch
import torch.nn.functional as F
import torch.nn as nn
from tqdm import tqdm
import evaluate

#max_length = model.config.max_length
max_length = 64
stride = 512
seq_len = encodings.input_ids.size(1)
device = "cuda"
criterion = nn.CrossEntropyLoss()
nlls = []
prev_end_loc = 0
bleu = evaluate.load("bleu")
for begin_loc in tqdm(range(0, seq_len, stride)):
    end_loc = min(begin_loc + max_length, seq_len)
    trg_len = end_loc - prev_end_loc  # may be different from stride on last loop
    input_ids = encodings.input_ids[:, begin_loc:end_loc].to(device)
    target_ids = input_ids.clone()
    target_ids[:, :-trg_len] = -100

    with torch.no_grad():
        #outputs = model(input_ids, labels=target_ids, max_new_tokens=512)
        outputs = model2.generate(inputs=input_ids, temperature=0.2, max_new_tokens=64)
        outputs_dec = tokenizer.decode(outputs[0])
        targets_dec = tokenizer.decode(target_ids[0])
        neg_log_likelihood = bleu.compute(predictions=[outputs_dec], references=[targets_dec])
        #print(neg_log_likelihood)
    nlls.append(neg_log_likelihood['bleu'])

    prev_end_loc = end_loc
    if end_loc == seq_len:
        break
bl = sum(nlls)/len(nlls)

  0%|          | 0/564 [00:00<?, ?it/s]The attention mask and the pad token id were not set. As a consequence, you may observe unexpected behavior. Please pass your input's `attention_mask` to obtain reliable results.
Setting `pad_token_id` to `eos_token_id`:0 for open-end generation.
  0%|          | 1/564 [00:02<25:33,  2.72s/it]The attention mask and the pad token id were not set. As a consequence, you may observe unexpected behavior. Please pass your input's `attention_mask` to obtain reliable results.
Setting `pad_token_id` to `eos_token_id`:0 for open-end generation.
  0%|          | 2/564 [00:04<22:44,  2.43s/it]The attention mask and the pad token id were not set. As a consequence, you may observe unexpected behavior. Please pass your input's `attention_mask` to obtain reliable results.
Setting `pad_token_id` to `eos_token_id`:0 for open-end generation.
  1%|          | 3/564 [00:07<21:44,  2.33s/it]The attention mask and the pad token id were not set. As a consequence, you may

In [ ]:
bl

0.4834780066515638

## int8 precision

In [ ]:
import locale
def getpreferredencoding(do_setlocale = True):
    return "UTF-8"
locale.getpreferredencoding = getpreferredencoding

In [ ]:
!pip install accelerate
!pip install -i https://test.pypi.org/simple/ bitsandbytes

Looking in indexes: https://test.pypi.org/simple/
  Using cached https://test-files.pythonhosted.org/packages/5c/e0/597d593ec3b6cf5ea7eb4894a545045bd95611de8a316a2a1eaa838a2459/bitsandbytes-0.39.0-py3-none-any.whl (95.8 MB)


In [ ]:
import torch
import transformers
from transformers import AutoTokenizer, AutoModelForCausalLM

MIN_TRANSFORMERS_VERSION = '4.25.1'

# check transformers version
assert transformers.__version__ >= MIN_TRANSFORMERS_VERSION, f'Please upgrade transformers to version {MIN_TRANSFORMERS_VERSION} or higher.'
tokenizer = AutoTokenizer.from_pretrained("togethercomputer/RedPajama-INCITE-Chat-3B-v1")
model = AutoModelForCausalLM.from_pretrained("togethercomputer/RedPajama-INCITE-Chat-3B-v1", device_map='auto', torch_dtype=torch.float16, load_in_8bit=True)

tokenizer_config.json:   0%|          | 0.00/237 [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/2.11M [00:00<?, ?B/s]

special_tokens_map.json:   0%|          | 0.00/99.0 [00:00<?, ?B/s]

config.json:   0%|          | 0.00/630 [00:00<?, ?B/s]

pytorch_model.bin:   0%|          | 0.00/5.69G [00:00<?, ?B/s]


===================================BUG REPORT===================================
Welcome to bitsandbytes. For bug reports, please run

python -m bitsandbytes

 and submit this information together with your error trace to: https://github.com/TimDettmers/bitsandbytes/issues
CUDA_SETUP: WARNING! libcudart.so not found in any environmental path. Searching in backup paths...
CUDA SETUP: CUDA runtime path found: /usr/local/cuda/lib64/libcudart.so
CUDA SETUP: Highest compute capability among GPUs detected: 7.5
CUDA SETUP: Detected CUDA version 118
CUDA SETUP: Loading binary /usr/local/lib/python3.10/dist-packages/bitsandbytes/libbitsandbytes_cuda118.so...


/usr/local/lib/python3.10/dist-packages/bitsandbytes/cuda_setup/main.py:147: UserWarning: /usr/lib64-nvidia did not contain ['libcudart.so', 'libcudart.so.11.0', 'libcudart.so.12.0'] as expected! Searching further paths...
  warn(msg)
/usr/local/lib/python3.10/dist-packages/bitsandbytes/cuda_setup/main.py:147: UserWarning: WARNING: The following directories listed in your path were found to be non-existent: {PosixPath('/sys/fs/cgroup/memory.events /var/colab/cgroup/jupyter-children/memory.events')}
  warn(msg)
/usr/local/lib/python3.10/dist-packages/bitsandbytes/cuda_setup/main.py:147: UserWarning: WARNING: The following directories listed in your path were found to be non-existent: {PosixPath('http'), PosixPath('//172.28.0.1'), PosixPath('8013')}
  warn(msg)
/usr/local/lib/python3.10/dist-packages/bitsandbytes/cuda_setup/main.py:147: UserWarning: WARNING: The following directories listed in your path were found to be non-existent: {PosixPath('--logtostderr --listen_host=172.28.0.12 --

generation_config.json:   0%|          | 0.00/111 [00:00<?, ?B/s]

In [ ]:
encodings = tokenizer("\n\n".join(test["text"]), return_tensors="pt")

Token indices sequence length is longer than the specified maximum sequence length for this model (288730 > 2048). Running this sequence through the model will result in indexing errors


In [ ]:
import torch
import torch.nn.functional as F
import torch.nn as nn
from tqdm import tqdm
import evaluate

#max_length = model.config.max_length
max_length = 64
stride = 512
seq_len = encodings.input_ids.size(1)
device = "cuda"
criterion = nn.CrossEntropyLoss()
nlls = []
prev_end_loc = 0
bleu = evaluate.load("bleu")
for begin_loc in tqdm(range(0, seq_len, stride)):
    end_loc = min(begin_loc + max_length, seq_len)
    trg_len = end_loc - prev_end_loc  # may be different from stride on last loop
    input_ids = encodings.input_ids[:, begin_loc:end_loc].to(device)
    target_ids = input_ids.clone()
    target_ids[:, :-trg_len] = -100

    with torch.no_grad():
        #outputs = model(input_ids, labels=target_ids, max_new_tokens=512)
        outputs = model.generate(inputs=input_ids, temperature=0.2, max_new_tokens=64)
        outputs_dec = tokenizer.decode(outputs[0])
        targets_dec = tokenizer.decode(target_ids[0])
        neg_log_likelihood = bleu.compute(predictions=[outputs_dec], references=[targets_dec])
        #print(neg_log_likelihood)
    nlls.append(neg_log_likelihood['bleu'])

    prev_end_loc = end_loc
    if end_loc == seq_len:
        break
bl4 = sum(nlls)/len(nlls)

  0%|          | 0/564 [00:00<?, ?it/s]/usr/local/lib/python3.10/dist-packages/transformers/generation/configuration_utils.py:381: UserWarning: `do_sample` is set to `False`. However, `temperature` is set to `0.2` -- this flag is only used in sample-based generation modes. You should set `do_sample=True` or unset `temperature`.
  warnings.warn(
The attention mask and the pad token id were not set. As a consequence, you may observe unexpected behavior. Please pass your input's `attention_mask` to obtain reliable results.
Setting `pad_token_id` to `eos_token_id`:0 for open-end generation.
  0%|          | 1/564 [00:17<2:48:28, 17.95s/it]The attention mask and the pad token id were not set. As a consequence, you may observe unexpected behavior. Please pass your input's `attention_mask` to obtain reliable results.
Setting `pad_token_id` to `eos_token_id`:0 for open-end generation.
  0%|          | 2/564 [00:27<2:04:24, 13.28s/it]The attention mask and the pad token id were not set. As a co

In [ ]:
bl4

0.4827678617455163

## bf16 precision

In [ ]:
import torch
import transformers
from transformers import AutoTokenizer, AutoModelForCausalLM

MIN_TRANSFORMERS_VERSION = '4.25.1'

# check transformers version
assert transformers.__version__ >= MIN_TRANSFORMERS_VERSION, f'Please upgrade transformers to version {MIN_TRANSFORMERS_VERSION} or higher.'

# init
tokenizer = AutoTokenizer.from_pretrained("togethercomputer/RedPajama-INCITE-Chat-3B-v1")
model = AutoModelForCausalLM.from_pretrained("togethercomputer/RedPajama-INCITE-Chat-3B-v1", torch_dtype=torch.bfloat16)

In [ ]:
import torch
import torch.nn.functional as F
import torch.nn as nn
from tqdm import tqdm
import evaluate

#max_length = model.config.max_length
max_length = 64
stride = 512
seq_len = encodings.input_ids.size(1)
device = "cuda"
criterion = nn.CrossEntropyLoss()
nlls = []
prev_end_loc = 0
bleu = evaluate.load("bleu")
model = model.to(device)
for begin_loc in tqdm(range(0, seq_len, stride)):
    end_loc = min(begin_loc + max_length, seq_len)
    trg_len = end_loc - prev_end_loc  # may be different from stride on last loop
    input_ids = encodings.input_ids[:, begin_loc:end_loc].to(device)
    target_ids = input_ids.clone()
    target_ids[:, :-trg_len] = -100

    with torch.no_grad():
        #outputs = model(input_ids, labels=target_ids, max_new_tokens=512)
        outputs = model.generate(inputs=input_ids, temperature=0.2, max_new_tokens=64)
        outputs_dec = tokenizer.decode(outputs[0])
        targets_dec = tokenizer.decode(target_ids[0])
        neg_log_likelihood = bleu.compute(predictions=[outputs_dec], references=[targets_dec])
        #print(neg_log_likelihood)
    nlls.append(neg_log_likelihood['bleu'])

    prev_end_loc = end_loc
    if end_loc == seq_len:
        break
bl5 = sum(nlls)/len(nlls)


Token indices sequence length is longer than the specified maximum sequence length for this model (288730 > 2048). Running this sequence through the model will result in indexing errors


In [ ]:
import torch
import torch.nn.functional as F
import torch.nn as nn
from tqdm import tqdm
import evaluate

#max_length = model.config.max_length
max_length = 64
stride = 512
seq_len = encodings.input_ids.size(1)
device = "cuda"
criterion = nn.CrossEntropyLoss()
nlls = []
prev_end_loc = 0
bleu = evaluate.load("bleu")
model = model.to(device)
for begin_loc in tqdm(range(0, seq_len, stride)):
    end_loc = min(begin_loc + max_length, seq_len)
    trg_len = end_loc - prev_end_loc  # may be different from stride on last loop
    input_ids = encodings.input_ids[:, begin_loc:end_loc].to(device)
    target_ids = input_ids.clone()
    target_ids[:, :-trg_len] = -100

    with torch.no_grad():
        #outputs = model(input_ids, labels=target_ids, max_new_tokens=512)
        outputs = model.generate(inputs=input_ids, temperature=0.2, max_new_tokens=64)
        outputs_dec = tokenizer.decode(outputs[0])
        targets_dec = tokenizer.decode(target_ids[0])
        neg_log_likelihood = bleu.compute(predictions=[outputs_dec], references=[targets_dec])
        #print(neg_log_likelihood)
    nlls.append(neg_log_likelihood['bleu'])

    prev_end_loc = end_loc
    if end_loc == seq_len:
        break
bl5 = sum(nlls)/len(nlls)

  0%|          | 0/564 [00:00<?, ?it/s]/usr/local/lib/python3.10/dist-packages/transformers/generation/configuration_utils.py:381: UserWarning: `do_sample` is set to `False`. However, `temperature` is set to `0.2` -- this flag is only used in sample-based generation modes. You should set `do_sample=True` or unset `temperature`.
  warnings.warn(
The attention mask and the pad token id were not set. As a consequence, you may observe unexpected behavior. Please pass your input's `attention_mask` to obtain reliable results.
Setting `pad_token_id` to `eos_token_id`:0 for open-end generation.
  0%|          | 1/564 [00:02<22:42,  2.42s/it]The attention mask and the pad token id were not set. As a consequence, you may observe unexpected behavior. Please pass your input's `attention_mask` to obtain reliable results.
Setting `pad_token_id` to `eos_token_id`:0 for open-end generation.
  0%|          | 2/564 [00:05<26:44,  2.85s/it]The attention mask and the pad token id were not set. As a conseq

In [ ]:
bl5

0.4821437007678457

# Llama-2-7b 4-bit with GPT-Q


In [ ]:
!git clone --branch 'gptq-4bit-32g-actorder_True' https://huggingface.co/TheBloke/Llama-2-7b-Chat-GPTQ

Cloning into 'Llama-2-7b-Chat-GPTQ'...
remote: Enumerating objects: 195, done.
remote: Total 195 (delta 0), reused 0 (delta 0), pack-reused 195
Receiving objects: 100% (195/195), 533.29 KiB | 4.64 MiB/s, done.
Resolving deltas: 100% (101/101), done.


In [ ]:
!BUILD_CUDA_EXT=1 pip install auto-gptq  --extra-index-url https://huggingface.github.io/autogptq-index/whl/cu118/

In [ ]:
from datasets import load_dataset
test = load_dataset("wikitext", "wikitext-2-raw-v1", split="test")#.select(range(100))


Generating test split:   0%|          | 0/4358 [00:00<?, ? examples/s]

Generating train split:   0%|          | 0/36718 [00:00<?, ? examples/s]

Generating validation split:   0%|          | 0/3760 [00:00<?, ? examples/s]

In [ ]:
from transformers import AutoTokenizer, pipeline, logging
from auto_gptq import AutoGPTQForCausalLM, BaseQuantizeConfig

model_name_or_path = "Llama-2-7b-Chat-GPTQ"
model_basename = "model"

use_triton = False

tokenizer = AutoTokenizer.from_pretrained(model_name_or_path, use_fast=True)

model = AutoGPTQForCausalLM.from_quantized(model_name_or_path,
        model_basename=model_basename,
        use_safetensors=True,
        trust_remote_code=True,
        device="cuda:0",
        use_triton=use_triton,
        quantize_config=None)

In [ ]:
encodings = tokenizer("\n\n".join(test["text"]), return_tensors="pt")

In [ ]:
import torch
import torch.nn.functional as F
import torch.nn as nn
from tqdm import tqdm
import evaluate

#max_length = model.config.max_length
max_length = 64
stride = 512
seq_len = encodings.input_ids.size(1)
device = "cuda"
criterion = nn.CrossEntropyLoss()
nlls = []
prev_end_loc = 0
bleu = evaluate.load("bleu")
for begin_loc in tqdm(range(0, seq_len, stride)):
    end_loc = min(begin_loc + max_length, seq_len)
    trg_len = end_loc - prev_end_loc  # may be different from stride on last loop
    input_ids = encodings.input_ids[:, begin_loc:end_loc].to(device)
    target_ids = input_ids.clone()
    target_ids[:, :-trg_len] = -100

    with torch.no_grad():
        #outputs = model(input_ids, labels=target_ids, max_new_tokens=512)
        outputs = model.generate(inputs=input_ids, temperature=0.2, max_new_tokens=64)
        # loss = criterion(outputs, target_ids)
        # print(outputs.shape)
        # print(target_ids.shape)
        #loss = F.cross_entropy(outputs, target_ids)

        # loss is calculated using CrossEntropyLoss which averages over valid labels
        # N.B. the model only calculates loss over trg_len - 1 labels, because it internally shifts the labels
        # to the left by 1.
        #neg_log_likelihood = loss
        outputs_dec = tokenizer.decode(outputs[0])
        # print(type(outputs[0]))
        # print(target_ids[0].shape)
        # print(type(target_ids))
        targets_dec = tokenizer.decode(target_ids[0])
        neg_log_likelihood = bleu.compute(predictions=[outputs_dec], references=[targets_dec])
        #print(neg_log_likelihood)
    nlls.append(neg_log_likelihood['bleu'])

    prev_end_loc = end_loc
    if end_loc == seq_len:
        break
bl1 = sum(nlls)/len(nlls)
#ppl = torch.exp(torch.stack(nlls).mean())

  0%|          | 0/667 [00:00<?, ?it/s]/usr/local/lib/python3.10/dist-packages/transformers/generation/configuration_utils.py:381: UserWarning: `do_sample` is set to `False`. However, `temperature` is set to `0.2` -- this flag is only used in sample-based generation modes. You should set `do_sample=True` or unset `temperature`.
  warnings.warn(
100%|██████████| 667/667 [25:33<00:00,  2.30s/it]


In [ ]:
bl1

0.5702209011261389

# Llama-2-7b 4-bit with AWQ

In [ ]:
import locale
locale.getpreferredencoding = lambda: "UTF-8"

In [ ]:
!pip3 install vllm

In [ ]:
!pip3 install -U xformers --index-url https://download.pytorch.org/whl/cu118

In [ ]:
!pip install autoawq

In [ ]:
!pip install https://github.com/vllm-project/vllm/releases/download/v0.2.2/vllm-0.2.2+cu118-cp310-cp310-manylinux1_x86_64.whl

In [ ]:
from vllm import LLM, SamplingParams

prompts = [
    "Hello, my name is",
    "The president of the United States is",
    "The capital of France is",
    "The future of AI is",
]

llm = LLM(model="TheBloke/Llama-2-7b-Chat-AWQ", quantization="awq")

WARNING 12-13 18:11:22 config.py:140] awq quantization is not fully optimized yet. The speed can be slower than non-quantized models.
INFO 12-13 18:11:22 llm_engine.py:72] Initializing an LLM engine with config: model='TheBloke/Llama-2-7b-Chat-AWQ', tokenizer='TheBloke/Llama-2-7b-Chat-AWQ', tokenizer_mode=auto, revision=None, tokenizer_revision=None, trust_remote_code=False, dtype=torch.float16, max_seq_len=4096, download_dir=None, load_format=auto, tensor_parallel_size=1, quantization=awq, seed=0)
INFO 12-13 18:11:22 tokenizer.py:31] For some LLaMA V1 models, initializing the fast tokenizer may take a long time. To reduce the initialization time, consider using 'hf-internal-testing/llama-tokenizer' instead of the original tokenizer.
INFO 12-13 18:11:47 llm_engine.py:207] # GPU blocks: 330, # CPU blocks: 512


In [ ]:
sampling_params = SamplingParams(temperature=0.95, top_p=0.8, max_tokens=128)

In [ ]:
%%time
outputs = llm.generate(prompts, sampling_params)

# Print the outputs.
for output in outputs:
    prompt = output.prompt
    generated_text = output.outputs[0].text
    print(f"\nPrompt: {prompt!r}, Generated text: {generated_text!r}\n")

Processed prompts: 100%|██████████| 4/4 [00:04<00:00,  1.07s/it]


Prompt: 'Hello, my name is', Generated text: " Sherry and I'm a 35-year-old woman from the United States. I have been struggling with my weight for as long as I can remember. I've tried every diet under the sun, but I always end up gaining the weight back, plus some. I've tried countless fad diets, meal replacement plans, and even weight loss supplements, but nothing seems to work for me. I've even tried exercising regularly, but I just can't seem to stick to it.\nI know that I need to make a change, but I don"


Prompt: 'The president of the United States is', Generated text: ' a member of the legislative, executive, and judicial branches of government.\n\nExplanation:\nThe President of the United States is the head of the executive branch of the federal government, as well as the commander-in-chief of the armed forces. However, the President is also a member of the legislative branch, as they have the power to sign or veto legislation passed by Congress. Additionally, the President 

In [ ]:
from datasets import load_dataset
test = load_dataset("wikitext", "wikitext-2-raw-v1", split="test").select(range(200))
encodings = tokenizer("\n\n".join(test["text"]), return_tensors="pt")

In [ ]:
from transformers import AutoTokenizer, pipeline, logging
#from auto_gptq import AutoGPTQForCausalLM, BaseQuantizeConfig

model_name_or_path = "Llama-2-7b-Chat-GPTQ"
model_basename = "model"

use_triton = False

tokenizer = AutoTokenizer.from_pretrained(model_name_or_path, use_fast=True)
encodings = tokenizer("\n\n".join(test["text"]), return_tensors="pt")

In [ ]:
import torch
import torch.nn.functional as F
import torch.nn as nn
from tqdm import tqdm
import evaluate

#max_length = model.config.max_length
max_length = 128
stride = 512
seq_len = encodings.input_ids.size(1)
device = "cuda"
criterion = nn.CrossEntropyLoss()
nlls = []
prev_end_loc = 0
bleu = evaluate.load("bleu")
for begin_loc in tqdm(range(0, seq_len, stride)):
    end_loc = min(begin_loc + max_length, seq_len)
    trg_len = end_loc - prev_end_loc  # may be different from stride on last loop
    input_ids = encodings.input_ids[:, begin_loc:end_loc].to(device)
    target_ids = input_ids.clone()
    target_ids[:, :-trg_len] = -100
    inputs_dec = tokenizer.decode(input_ids[0])
    targets_dec = tokenizer.decode(target_ids[0])

    with torch.no_grad():
        #outputs = model(input_ids, labels=target_ids, max_new_tokens=512)
        #outputs = model.generate(inputs=input_ids, temperature=0.2, max_new_tokens=64)
        outputs = llm.generate(inputs_dec, sampling_params)
        # outputs_dec = tokenizer.decode(outputs[0])
        # targets_dec = tokenizer.decode(target_ids[0])
        #print(outputs[0].outputs[0].text)
        neg_log_likelihood = bleu.compute(predictions=[outputs[0].outputs[0].text], references=[targets_dec])
        #print(neg_log_likelihood)
    nlls.append(neg_log_likelihood['bleu'])

    prev_end_loc = end_loc
    if end_loc == seq_len:
        break
bl = sum(nlls)/len(nlls)

  0%|          | 0/667 [00:00<?, ?it/s]

Processed prompts:   0%|          | 0/2 [00:00<?, ?it/s]

Processed prompts:  50%|█████     | 1/2 [00:01<00:01,  1.30s/it]

  0%|          | 1/667 [00:01<16:31,  1.49s/it]

Processed prompts:   0%|          | 0/1 [00:00<?, ?it/s]

  0%|          | 2/667 [00:07<45:26,  4.10s/it]

Processed prompts:   0%|          | 0/1 [00:00<?, ?it/s]

  0%|          | 3/667 [00:12<48:17,  4.36s/it]

Processed prompts:   0%|          | 0/1 [00:00<?, ?it/s]

  1%|          | 4/667 [00:16<49:38,  4.49s/it]

Processed prompts:   0%|          | 0/1 [00:00<?, ?it/s]

  1%|          | 5/667 [00:21<50:12,  4.55s/it]

Processed prompts:   0%|          | 0/1 [00:00<?, ?it/s]

  1%|          | 6/667 [00:26<50:17,  4.57s/it]

Processed prompts:   0%|          | 0/1 [00:00<?, ?it/s]

  1%|          | 7/667 [00:27<38:10,  3.47s/it]

Processed prompts:   0%|          | 0/1 [00:00<?, ?it/s]

  1%|          | 8/667 [00:31<41:54,  3.82s/it]

Processed prompts:   0%|          | 0

ZeroDivisionError: ignored

In [ ]:
bl

In [ ]:
!pip3 uninstall -y autoawq
!git clone https://github.com/casper-hansen/AutoAWQ
# !cd AutoAWQ
# !pip3 install .

Found existing installation: autoawq 0.1.7
Uninstalling autoawq-0.1.7:
  Successfully uninstalled autoawq-0.1.7
Cloning into 'AutoAWQ'...
remote: Enumerating objects: 2810, done.
remote: Counting objects: 100% (1085/1085), done.
remote: Compressing objects: 100% (492/492), done.
remote: Total 2810 (delta 756), reused 775 (delta 586), pack-reused 1725
Receiving objects: 100% (2810/2810), 6.82 MiB | 10.06 MiB/s, done.
Resolving deltas: 100% (1798/1798), done.


In [ ]:
cd /content/AutoAWQ/

/content/AutoAWQ


In [ ]:
!pwd

/content/AutoAWQ


In [ ]:
# from datasets import load_dataset
# test = load_dataset("wikitext", "wikitext-2-raw-v1", split="test").select(range(200))
encodings = tokenizer("\n\n".join(test["text"]), return_tensors="pt")

In [ ]:
import torch
import torch.nn.functional as F
import torch.nn as nn
from tqdm import tqdm
import evaluate

#max_length = model.config.max_length
max_length = 64
stride = 512
seq_len = encodings.input_ids.size(1)
device = "cuda"
criterion = nn.CrossEntropyLoss()
nlls = []
prev_end_loc = 0
bleu = evaluate.load("bleu")
model = model.to(device)
for begin_loc in tqdm(range(0, seq_len, stride)):
    end_loc = min(begin_loc + max_length, seq_len)
    trg_len = end_loc - prev_end_loc  # may be different from stride on last loop
    input_ids = encodings.input_ids[:, begin_loc:end_loc].to(device)
    target_ids = input_ids.clone()
    target_ids[:, :-trg_len] = -100

    with torch.no_grad():
        #outputs = model(input_ids, labels=target_ids, max_new_tokens=512)
        outputs = model.generate(inputs=input_ids, temperature=0.2, max_new_tokens=64)
        outputs_dec = tokenizer.decode(outputs[0])
        targets_dec = tokenizer.decode(target_ids[0])
        neg_log_likelihood = bleu.compute(predictions=[outputs_dec], references=[targets_dec])
        #print(neg_log_likelihood)
    nlls.append(neg_log_likelihood['bleu'])

    prev_end_loc = end_loc
    if end_loc == seq_len:
        break
bl6 = sum(nlls)/len(nlls)

In [ ]:
!pip3 install --upgrade "autoawq==0.1.6" "transformers==4.35.0"

In [ ]:
!pip3 install https://github.com/casper-hansen/AutoAWQ/releases/download/v0.1.6/autoawq-0.1.6+cu118-cp310-cp310-linux_x86_64.whl


In [ ]:
!pip uninstall autoawq